In [1]:
import sys
import subprocess
import re
import os
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cmath
import operator
import random
from operator import itemgetter, attrgetter
import matplotlib.cm as cm
import vtk

## import MI measures
from mi_functions import *

In [2]:
totPercentage = 9

In [3]:
# isabel
file1 = '../Data/Isabel_vti/isabel_qgraup_25.vti'
file2 = '../Data/Isabel_vti/isabel_qcloud_25.vti'
file3 = '../Data/Isabel_vti/isabel_precip_25.vti'

arrayName1 = 'QGraup'
arrayName2 = 'QCloud'
arrayName3 = 'Precipitation'
data_set = 'isabel'
samp_type = 'pmi' # random, pmi

In [4]:
# # Asteroid
# file1 = '../Data/Asteroid/tev.vti'
# file2 = '../Data/Asteroid/v02.vti'
# file3 = '../Data/Asteroid/v03.vti'

# arrayName1 = 'tev'
# arrayName2 = 'v02'
# arrayName3 = 'v03'
# data_set = 'asteroid'
# samp_type = 'pmi' # random, pmi

In [5]:
# ## load vti data
R1 = vtk.vtkXMLImageDataReader()
R1.SetFileName(file1)
R1.Update()
dataArray1 = R1.GetOutput().GetPointData().GetArray(arrayName1)

R2 = vtk.vtkXMLImageDataReader()
R2.SetFileName(file2)
R2.Update()
dataArray2 = R2.GetOutput().GetPointData().GetArray(arrayName2)

R3 = vtk.vtkXMLImageDataReader()
R3.SetFileName(file3)
R3.Update()
dataArray3 = R3.GetOutput().GetPointData().GetArray(arrayName3)

var1=np.zeros(dataArray1.GetNumberOfTuples()) 
var2=np.zeros(dataArray2.GetNumberOfTuples()) 
var3=np.zeros(dataArray3.GetNumberOfTuples())

var_highdim=np.zeros((dataArray3.GetNumberOfTuples(),3))
print np.shape(var_highdim)

for i in range(dataArray1.GetNumberOfTuples()):
    var1[i] = dataArray1.GetTuple1(i)
    var2[i] = dataArray2.GetTuple1(i)
    var3[i] = dataArray3.GetTuple1(i)
    var_highdim[i] = [var1[i],var2[i],var3[i]]

dims = R1.GetOutput().GetDimensions()

min_var1 = np.min(var1)
max_var1 = np.max(var1)

min_var2 = np.min(var2)
max_var2 = np.max(var2)

min_var3 = np.min(var3)
max_var3 = np.max(var3)

print R1.GetOutput().GetSpacing()
print R1.GetOutput().GetOrigin()

(3125000, 3)
(1.0, 1.0, 1.0)
(0.0, 0.0, 0.0)


In [6]:
if samp_type == 'pmi':
    mixinig_factor_PMI = 1
elif samp_type == 'random':
    mixinig_factor_PMI = 0
    
percentageToStroe = totPercentage*mixinig_factor_PMI
percentage_rand = totPercentage*(1-mixinig_factor_PMI)
pointsToretain = int((len(var1)*percentageToStroe)/100.00)
print 'Percentage needed: ' + str(totPercentage)
print 'Points to be taken PMI based: ' + str(pointsToretain)
print 'Points to be taken randomly: ' + str(int((len(var1)*percentage_rand)/100.00))

outfile = '../output/joint_'+samp_type+'_sampled_'+data_set+'_'+arrayName1+'_'+arrayName2+'_'\
+arrayName3+'_'+str(totPercentage)+'.vtp'
pmivol_file = '../output/pmi_vol_p_vel.vti'

Percentage needed: 9
Points to be taken PMI based: 281250
Points to be taken randomly: 0


In [7]:
## compute histogram and PMI
numBins=128
numSamples = np.shape(var1)[0]

hist1 = np.histogram(var1,bins=numBins)
hist2 = np.histogram(var2,bins=numBins)
hist3 = np.histogram(var3,bins=numBins)
hist3D = np.histogramdd(var_highdim,bins=numBins)

Array1 = hist1[0]
Array2 = hist2[0]
ArrayComb = hist3D[0]

## compute PMI
PMI = compute_pointwise_mutual_information_3d(Array1,Array2,Array2,ArrayComb,numSamples,numBins)

#PMI = abs(PMI)

max_pmi = np.max(PMI)
min_pmi = np.min(PMI)
print 'min pmi: ' + str(min_pmi)
print 'max pmi: ' + str(max_pmi)

# normalize PMI
PMI = (PMI-min_pmi)/(max_pmi-min_pmi)
max_pmi = np.max(PMI)
min_pmi = np.min(PMI)

PMI = np.power(PMI,14) #### 15 for asteroid, 8 for isabel P and qva
print np.shape(PMI)

min pmi: -21.50170159798641
max pmi: 28.242832257134996
(128, 128, 128)


In [8]:
def compute_3d_to_1d_map (pts, dims):
    return pts[0] + dims[0]*(pts[1]+dims[1]*pts[2])

######################################################################
def sample_pmi_based(PMI,data1,data2,data3,arrayName1,arrayName2,arrayName3):
    
    numBins = len(PMI[0])
    
    numPts = data1.GetPointData().GetArray(arrayName1).GetNumberOfTuples()
    dataArr1 = data1.GetPointData().GetArray(arrayName1)
    dataArr2 = data2.GetPointData().GetArray(arrayName2)
    dataArr3 = data3.GetPointData().GetArray(arrayName3)
    
    range_vals1 = dataArr1.GetRange() 
    range_vals2 = dataArr2.GetRange()
    range_vals3 = dataArr3.GetRange()
    
    pdata = vtk.vtkPolyData()
    pts_arr = vtk.vtkPoints()
    sampled_arr = vtk.vtkDoubleArray()
    sampled_arr.SetName(arrayName1)
    
    sampled_arr1 = vtk.vtkDoubleArray()
    sampled_arr1.SetName(arrayName2)
    
    sampled_arr3 = vtk.vtkDoubleArray()
    sampled_arr3.SetName(arrayName3)
    
    for i in range(numPts):
        randval = random.uniform(0, 1)
        
        dataval1 = dataArr1.GetTuple1(i)
        bin_id1 = int(((dataval1 - range_vals1[0])/(range_vals1[1]-range_vals1[0]))*(numBins-1))
        
        dataval2 = dataArr2.GetTuple1(i)
        bin_id2 = int(((dataval2 - range_vals2[0])/(range_vals2[1]-range_vals2[0]))*(numBins-1))
        
        dataval3 = dataArr3.GetTuple1(i)
        bin_id3 = int(((dataval3 - range_vals3[0])/(range_vals3[1]-range_vals3[0]))*(numBins-1))
        
        if randval <= PMI[bin_id1][bin_id2][bin_id3]:
            pts_arr.InsertNextPoint(data1.GetPoint(i))
            sampled_arr.InsertNextTuple1(dataval1)
            sampled_arr1.InsertNextTuple1(dataval2)
            sampled_arr3.InsertNextTuple1(dataval3)
    
    pdata.SetPoints(pts_arr)
    pdata.GetPointData().AddArray(sampled_arr)
    pdata.GetPointData().AddArray(sampled_arr1)
    pdata.GetPointData().AddArray(sampled_arr3)
    
    return pdata

####################################################################
def insert_corner_pts(data,arrayName1,data1,arrayName2,data3,arrayName3,sampled_data):
    
    dims = data.GetDimensions()
    varArr = data.GetPointData().GetArray(arrayName1)
    varArr1 = data1.GetPointData().GetArray(arrayName2)
    varArr3 = data3.GetPointData().GetArray(arrayName3)
    
    pts = [0,0,0]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    var_val3 = varArr3.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    sampled_data.GetPointData().GetArray(arrayName3).InsertNextTuple1(var_val3)
    
    pts = [0,dims[1]-1,0]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    var_val3 = varArr3.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    sampled_data.GetPointData().GetArray(arrayName3).InsertNextTuple1(var_val3)
    
    pts = [0,0,dims[2]-1]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    var_val3 = varArr3.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    sampled_data.GetPointData().GetArray(arrayName3).InsertNextTuple1(var_val3)
    
    pts = [0,dims[1]-1,dims[2]-1]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    var_val3 = varArr3.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    sampled_data.GetPointData().GetArray(arrayName3).InsertNextTuple1(var_val3)
    
    pts = [dims[0]-1,0,0]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    var_val3 = varArr3.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    sampled_data.GetPointData().GetArray(arrayName3).InsertNextTuple1(var_val3)
    
    pts = [dims[0]-1,dims[1]-1,0]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    var_val3 = varArr3.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    sampled_data.GetPointData().GetArray(arrayName3).InsertNextTuple1(var_val3)
    
    pts = [dims[0]-1,0,dims[2]-1]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    var_val3 = varArr3.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    sampled_data.GetPointData().GetArray(arrayName3).InsertNextTuple1(var_val3)
    
    pts = [dims[0]-1,dims[1]-1,dims[2]-1]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    var_val3 = varArr3.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    sampled_data.GetPointData().GetArray(arrayName3).InsertNextTuple1(var_val3)
    
    return sampled_data

######################################################################
def sample_random(data,arrayName1,data1,arrayName2,data3,arrayName3,percentage,sampled_data):
    
    totPts = data.GetNumberOfPoints()
    varArr = data.GetPointData().GetArray(arrayName1)
    varArr1 = data1.GetPointData().GetArray(arrayName2)
    varArr3 = data3.GetPointData().GetArray(arrayName3)
    numOfPtsNeeded = int((totPts*percentage)/100.0)
    
    print 'Number of randomly selected points: ' + str(numOfPtsNeeded)
    
    rand_ids = np.random.randint(totPts, size=numOfPtsNeeded)
    
    for i in range(len(rand_ids)):
        pts = data.GetPoint(rand_ids[i])
        var_val = varArr.GetTuple1(rand_ids[i])
        var_val1 = varArr1.GetTuple1(rand_ids[i])
        var_val3 = varArr3.GetTuple1(rand_ids[i])
        
        
        sampled_data.GetPoints().InsertNextPoint(pts)
        sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
        sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
        sampled_data.GetPointData().GetArray(arrayName3).InsertNextTuple1(var_val3)
        
    return sampled_data

######################################################################
def write_vtp(filename,pdata):
    pdatawriter = vtk.vtkXMLPolyDataWriter()
    pdatawriter.SetFileName(filename)
    pdatawriter.SetInputData(pdata)
    pdatawriter.Write()

In [9]:
ptsToCome = np.sum(PMI*ArrayComb)
print int(ptsToCome)

if ptsToCome < pointsToretain:
    fraction = pointsToretain/float(ptsToCome)
    PMI_mod = PMI*fraction
else:
    fraction = ptsToCome/float(pointsToretain)
    PMI_mod = PMI/fraction
    
print 'fraction: ' + str(fraction)

ptsToCome = np.sum(PMI_mod*ArrayComb)
print int(ptsToCome)

305
fraction: 921.742079824
281250


In [10]:
## Sample based on importance function
sampled_data = sample_pmi_based(PMI_mod,R1.GetOutput(),R2.GetOutput(),R3.GetOutput(),arrayName1,arrayName2,arrayName3)

percentagePickedPMI = 100*(sampled_data.GetNumberOfPoints()/float(len(var1)))
percentage_diff = (totPercentage - percentagePickedPMI)

print 'Percentage_diff: ' + str(percentage_diff)
if percentage_diff > 0:
    sampled_data_mixed = sample_random(R1.GetOutput(),arrayName1,R2.GetOutput(),arrayName2,R3.GetOutput(),arrayName3,percentage_diff,sampled_data)
else:
    sampled_data_mixed = sample_random(R1.GetOutput(),arrayName1,R2.GetOutput(),arrayName2,R3.GetOutput(),arrayName3,percentage_rand,sampled_data)
    
## add the 8 corner points for linear interpolation to work
sampled_data_final = insert_corner_pts(R1.GetOutput(),arrayName1,R2.GetOutput(),arrayName2,R3.GetOutput(),arrayName3,sampled_data_mixed)

Percentage_diff: 6.851008
Number of randomly selected points: 214094


In [11]:
print 'Final number of sampled points picked is: '+str(sampled_data_final.GetNumberOfPoints())+' and percentage: '+ \
str(100*(sampled_data_final.GetNumberOfPoints()/float(len(var1))))

## write the data out
print outfile
write_vtp(outfile,sampled_data_final)

Final number of sampled points picked is: 281258 and percentage: 9.000256
../output/joint_pmi_sampled_isabel_QGraup_QCloud_Precipitation_9.vtp
